In [1]:
# Imports

%matplotlib nbagg
# %matplotlib osx

import os
import re
import shutil

import spectral
import matplotlib
import matplotlib.pyplot as plt
from tifffile import imsave
import numpy as np
from tkinter.filedialog import askdirectory, askopenfilenames
from tkinter import Tk

import general_funcs

In [6]:
# Extract all thermal images taken by thermal sensor mounted to headwall sensor

# Tk().withdraw()
input_folder = askdirectory(title='Select headwall thermal data folder')
file_list = os.listdir(input_folder)
regex = re.compile('raw_\d*\.hdr')
filtered_file_list = [i for i in file_list if regex.search(i)]
filtered_file_list.sort(key=lambda file_name: int(re.split('_|\.', file_name)[1]))
print('File folder:', input_folder)
print('Files used:', filtered_file_list)

File folder: /Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Downloads
Files used: ['raw_0.hdr', 'raw_2000.hdr', 'raw_4000.hdr', 'raw_6000.hdr', 'raw_8000.hdr']


In [32]:
output_folder = askdirectory(title='Choose output folder')
flag = 0
for file_name in filtered_file_list:
    file = os.path.join(input_folder, file_name)
    img_chunk = spectral.open_image(file).asarray()
    img_chunk = np.einsum('ijk->kij', img_chunk)
    print(img_chunk.shape)
    output_file = os.path.join(output_folder, file_name + '_test.tiff')
    imsave(output_file, img_chunk)

(272, 2000, 640)
(272, 2000, 640)
(272, 2000, 640)
(272, 2000, 640)
(272, 1180, 640)


In [21]:
output_file

'/Users/lj/Downloads/raw_0.hdr_test.tiff'

In [18]:
output_folder = askdirectory(title='Choose output folder')
img_flag = 0
for file_name in filtered_file_list:
    img_chunk = spectral.open_image(os.path.join(input_folder, file_name))
    h, w, d = img_chunk.shape
    for i in range(img_flag, img_flag+d):
        layer_i = img_chunk[:, :, i-img_flag]
        layer_i = np.squeeze(layer_i)
        imsave(os.path.join(output_folder, str(i) + '.tiff'), layer_i)
    img_flag += d
print(img_flag, 'files generated')

# Use one out of N imgs from all data
N = 5
percentage_str = str("{:.0%}".format(1/N))
output_folder_partial = output_folder+' - '+percentage_str
if not os.path.exists(output_folder_partial):
    os.mkdir(output_folder_partial)

data_list = os.listdir(output_folder)
for i in range(0, len(data_list), N):
    shutil.copy(os.path.join(output_folder, str(i)+'.tiff'), os.path.join(output_folder_partial, str(i)+'.tiff'))
    
general_funcs.done()

1360 files generated


FileNotFoundError: [Errno 2] No such file or directory: '/Users/lj/Downloads/1360.tiff'